In [1]:
import numpy as np
from tensorflow.keras.models import Model 
from keras import backend as K
from tensorflow.keras.layers import Conv2D,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.layers import DepthwiseConv2D,Activation,Input,Reshape,Add,Dropout
import tensorflow.keras as tfk

In [2]:
def ReLU6(x):
    return K.relu(x,max_value=6.0)

In [3]:
def _conv_block(inputs,filters,kernel,s):
    """
    inputs - the input tensor
    filters - integer, number of filters
    kernel - integer/tuple (,) of integers for the kernel size
    s - integer/tuple(,) of integers for strides 
    
    return - the output tensor 
    """
    
    x = Conv2D(filters,kernel,strides=s,padding='same')(inputs)
    x = BatchNormalization()(x)
    return Activation(ReLU6)(x)

In [4]:
def _bottleneck_block(inputs,s,filters,ex_factor,skip = False):

    exp = ex_factor * inputs.shape[-1]
    
    
    x = _conv_block(inputs=inputs,filters=exp,kernel=(1,1),s=(1,1))
    
    x = DepthwiseConv2D((3,3), strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(ReLU6)(x)
    
    x = Conv2D(filters=filters,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x = BatchNormalization()(x)
 
    if (skip):
        x = Add()([x, inputs])
    
    return x
    

In [5]:
 def _residual_block(inputs,n,filters,exp,s):
        
        x = _bottleneck_block(inputs=inputs,s=s,filters=filters,ex_factor=exp)
        for i in range(1,n):
            x = _bottleneck_block(inputs=x,s=1,filters=filters,ex_factor=exp,skip=True)
        return x

In [6]:
def MobileNet_V2(input_shape,classes):
    
    inputs = Input(shape=input_shape)
    
    x = _conv_block(inputs=inputs,filters=32,kernel=(3,3),s=2)
    x = _residual_block(inputs=x,n=1,filters=16,exp=1,s=1)
    x = _residual_block(inputs=x,n=2,filters=24,exp=6,s=2)
    x = _residual_block(inputs=x,n=3,filters=32,exp=6,s=2)
    x = _residual_block(inputs=x,n=4,filters=64,exp=6,s=2)
    x = _residual_block(inputs=x,n=3,filters=96,exp=6,s=1)
    x = _residual_block(inputs=x,n=3,filters=160,exp=6,s=2)
    x = _residual_block(inputs=x,n=1,filters=320,exp=6,s=1)
    
    x = _conv_block(inputs=x,filters=1280,kernel=(1,1),s=1)
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, 1280))(x)
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(classes, (1, 1), padding='same')(x)
    x = Activation('softmax', name='softmax')(x)
    output = Reshape((classes,))(x)
    
    model = Model(inputs,output)
    return model

In [7]:
model = MobileNet_V2((300,300,3),100)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 150, 150, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 150, 150, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 150, 150, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________